<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/model_experiments/model_slices.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/model_experiments/model_slices.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Model Slices

Model Slice functions similarly to Catalog Slice, with both essentially being saved searches. However, there are key differences in their functionalities. While Catalog Slice searches within a specific data catalog, Model Slice extends its data row search across a model run in a model. You can construct a Model Slice by using one or more filters to curate a collection of data rows. Often users will combine filters to surface high-impact data and then save the results as a Model Slice.

This notebook is used to go over some common Labelbox SDK methods to interact with Model Slices created through the Labelbox platform.

See [Slices](https://docs.labelbox.com/docs/slices-1) for more information on modifying Model Slices.

## Set up

In [ ]:
%pip install -q --upgrade "labelbox[data]"

In [1]:
import labelbox as lb
import uuid

## API key and client
Provide a valid API key below to connect to the Labelbox client properly. For more information, please review the [Create API key](https://docs.labelbox.com/reference/create-api-key) guide.

In [ ]:
API_KEY = None
client = lb.Client(api_key=API_KEY)

## Create Model Slice

In order to interact with model slices, you must create a Model Experiment with a Model Run and then create a Model Slice through the platform. The steps below go over this process. See [Model](https://docs.labelbox.com/reference/model) from our developer guides for more information.

### Creating Model Experiment

To create a Model Experiment you will need to create an ontology. See [Ontology](https://docs.labelbox.com/reference/ontology) for more information

#### Ontology

In [ ]:
classification_features = [
    lb.Classification(
        class_type=lb.Classification.Type.CHECKLIST,
        name="Quality Issues",
        options=[
            lb.Option(value="blurry", label="Blurry"),
            lb.Option(value="distorted", label="Distorted"),
        ],
    )
]

ontology_builder = lb.OntologyBuilder(tools=[],
                                      classifications=classification_features)

ontology = client.create_ontology(
    "Ontology from new features",
    ontology_builder.asdict(),
    media_type=lb.MediaType.Image,
)

#### Model Experiment

In [ ]:
model = client.create_model(name=f"Model Slice Demo {str(uuid.uuid4())}", ontology_id=ontology.uid)

### Creating a Model Run from Model Experiment

On this step we will need to create a dataset to attach data rows to our model run. See [Dataset](https://docs.labelbox.com/reference/dataset) for more information.

#### Dataset and Data Rows

In [ ]:
# send a sample image as data row for a dataset
global_key = str(uuid.uuid4())

test_img_url = {
    "row_data":
        "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg",
    "global_key":
        global_key,
}

dataset = client.create_dataset(name="foundry-demo-dataset")
task = dataset.create_data_rows([test_img_url])
task.wait_till_done()

print(f"Errors: {task.errors}")
print(f"Failed data rows: {task.failed_data_rows}")

#### Model Run and Attach Data Rows

In [ ]:
model_run_name = "Model Slice Demo"
example_config = {
    "learning_rate": 0.001,
    "batch_size": 32,
}
model_run = model.create_model_run(name=model_run_name, config=example_config)

In [ ]:
model_run.upsert_data_rows(global_keys=[global_key])

### Make Model Slice Inside UI

Creating a Model Slice is currently not supported through the SDK, however, to showcase how to interact with Model Slice, we are going to generate a Model Slice through the UI.

#### Workflow

1. Navigate to ***Model*** section of the Labelbox Platform, select the ***Experiment*** type, and select the Model Experiment that was created.
2. You must have a filter created in order to save a slice. For this example, click ***Search your data*** dropdown and then ***Data row***.
3. Change ***is one of*** dropdown to ***is not one of*** then type "test" into the ***Search for an id*** search box.
4. Hit ***Enter*** and select ***Save slice***.
5. Give the slice a name and select ***Save***.
6. Above the ***Search your data*** dropdown you will see your slice's name. Select that dropdown and click ***Copy slice ID***.
7. Paste the ***Slice ID*** below.

In [ ]:
SLICE_ID = ""

### Get Model Slice

In [ ]:
model_slice = client.get_model_slice(SLICE_ID)

### Obtain Data Row IDs from Model Slice

In [ ]:
data_row_ids = model_slice.get_data_row_ids(model_run.uid)

for data_row_id in data_row_ids:
    print(data_row_id)

### Obtain Data Row Identifiers Objects

In [ ]:
data_rows = model_slice.get_data_row_identifiers(model_run.uid)

for data_row in data_rows:
    print(data_row)

### Model Slice Attributes

In [ ]:
# name (str)
model_slice.name

# description (str)
model_slice.description

# updated at (datetime)
model_slice.updated_at

# created at (datetime)
model_slice.created_at

# filter (list[dict])
model_slice.filter

## Clean up
Uncomment and run the cell below to optionally delete Labelbox objects created.

In [ ]:
# model_run.delete()
# model.delete()
# dataset.delete()